# TTC Delay Data Preprocessing - Task Template

**Goal:** Clean and preprocess delay data from 2024 and 2025 for Phase 1 modeling

**Input Files:**
- `ttc-subway-delay-2024.xlsx`
- `ttc-subway-delay-data-since-2025.csv`

**Output File:**
- `cleaned_ttc_delay_data.csv`

---

## Setup & Imports

Import required libraries and load both datasets.

In [ ]:
# TODO: Import pandas, numpy, and any other libraries you need
# Hint: You'll need pandas for data manipulation

In [ ]:
# TODO: Load 2024 Excel file
# Hint: Use pd.read_excel() with sheet_name='Subway'
# Store in variable: df_2024

In [ ]:
# TODO: Load 2025 CSV file
# Hint: Use pd.read_csv()
# Store in variable: df_2025

In [ ]:
# TODO: Combine both dataframes
# Hint: Use pd.concat() with ignore_index=True
# Store in variable: df (this will be your main dataframe)

In [ ]:
# TODO: Inspect the combined dataframe
# Hint: Check shape, columns, first few rows, data types, info()

---

## Task 1: Standardize Line Names

**Objective:** Map all inconsistent line names to canonical forms (Line 1, Line 2, Line 4)

**Current Issues:**
- YU, YU/BD, YU / BD, YU/ BD, LINE 1 → Should all be "Line 1"
- BD, BLOOR DANFORTH, BD/YU, BD / YU → Should all be "Line 2"
- SHP, SHEP → Should be "Line 4"
- Invalid: 109 RANEE, 20 CLIFFSIDE, TRACK LEVEL ACTIVITY → Should be removed

**Hints:**
1. First, check `df['Line'].unique()` to see all variations
2. Create a mapping dictionary with old name → new name
3. Use `.map()` or `.replace()` to apply the mapping
4. For multi-line entries (YU/BD), decide: keep as "Line 1/2" or drop
5. Verify after mapping: check unique values again

In [ ]:
# TODO: Examine current Line values
# Print unique values and counts
# Hint: df['Line'].unique() and df['Line'].value_counts()

In [ ]:
# TODO: Create mapping dictionary
# Map all variations to canonical names
# Example structure:
# line_mapping = {
#     'YU': 'Line 1',
#     'YU/BD': 'Line 1/2',
#     ... (add all variations)
# }
# Hint: Reference the task guide for all 22 variations

In [ ]:
# TODO: Apply mapping to Line column
# Hint: Use df['Line'].map(line_mapping) or df['Line'].replace(line_mapping)

In [ ]:
# TODO: Verify mapping worked
# Check unique values after mapping
# Should see only: Line 1, Line 2, Line 4, Line 1/2, etc. (canonical forms)

---

## Task 2: Remove Invalid Records

**Objective:** Remove rows with missing or erroneous Line values

**Records to Remove:**
- ~44-67 rows with null/NaN Line values
- Erroneous entries like "109 RANEE", "20 CLIFFSIDE", "TRACK LEVEL ACTIVITY"

**Hints:**
1. Check `df['Line'].isnull().sum()` to count nulls
2. Use `df.dropna(subset=['Line'])` to remove null rows
3. Use `df[~df['Line'].isin(['list', 'of', 'invalid', 'values'])]` to filter invalid entries
4. Log record counts before and after to confirm removal
5. Expected result: ~52,000-52,064 records remaining

In [ ]:
# TODO: Check for null Line values
# Print count of nulls and percentage
# Hint: df['Line'].isnull().sum()

In [ ]:
# TODO: Remove rows with null Line
# Store record count before and after
# Hint: Use .dropna() or boolean indexing

In [ ]:
# TODO: Remove erroneous entries
# Create list of invalid values to remove
# Hint: Use df[~df['Line'].isin(['invalid', 'entries'])]

In [ ]:
# TODO: Log results
# Print: Original record count, Records removed, Final count

---

## Task 3: Parse Time Column

**Objective:** Extract hour from Time column (HH:MM format → integer 0-23)

**Current Format:** Text like "02:00", "08:46", "17:05"

**Hints:**
1. Create function to parse time strings
2. Split on ":" character and take first part
3. Convert to integer
4. Use `.apply()` to apply function to entire column
5. Verify: all values should be 0-23, no nulls
6. Drop original Time column after extraction

In [ ]:
# TODO: Examine Time column
# Check sample values, data type, unique counts
# Hint: df['Time'].head(10), df['Time'].dtype

In [ ]:
# TODO: Create function to extract hour
# Function should:
# - Take time string as input
# - Split on ':'
# - Return first part as integer
# - Handle errors gracefully
# Hint: def extract_hour(time_str):

In [ ]:
# TODO: Create 'hour' column
# Apply function to Time column
# Hint: df['hour'] = df['Time'].apply(extract_hour)

In [ ]:
# TODO: Validate hour values
# Check: min, max, data type, any nulls
# Should be: min=0, max=23, type=int, nulls=0

In [ ]:
# TODO: Drop Time column (no longer needed)
# Hint: df.drop(columns=['Time'])

---

## Task 4: Parse Date Column

**Objective:** Ensure Date is datetime format and extract temporal features (year, month, week)

**Current State:**
- 2024 file: already datetime
- 2025 file: text format "2025-01-01"

**Hints:**
1. Check current data type: `df['Date'].dtype`
2. Convert to datetime: `pd.to_datetime(df['Date'])`
3. Extract year: `df['Date'].dt.year`
4. Extract month: `df['Date'].dt.month`
5. Extract week: `df['Date'].dt.isocalendar().week`
6. Verify: year should be 2024 or 2025, month 1-12, week 1-53

In [ ]:
# TODO: Check current Date format
# Check data type and sample values
# Hint: df['Date'].dtype, df['Date'].head()

In [ ]:
# TODO: Convert Date to datetime
# Hint: df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# TODO: Extract year
# Create column 'year' with values 2024 or 2025
# Hint: df['year'] = df['Date'].dt.year

In [ ]:
# TODO: Extract month
# Create column 'month' with values 1-12
# Hint: df['month'] = df['Date'].dt.month

In [ ]:
# TODO: Extract week
# Create column 'week' with values 1-53
# Hint: df['week'] = df['Date'].dt.isocalendar().week

In [ ]:
# TODO: Validate temporal columns
# Check ranges: year (2024-2025), month (1-12), week (1-53)
# Check for any nulls

---

## Task 5: Define Target Variable

**Objective:** Create binary 'is_delayed' column (0=on-time, 1=delayed)

**Decision:** What threshold defines "delayed"?
- **Recommended: 5 minutes**
- Alternative: 10 minutes

**Hints:**
1. Analyze Min Delay distribution first (percentiles, mean, median, max)
2. Create rule: if Min Delay >= threshold, then 1, else 0
3. Check class balance (should be roughly 70% on-time, 30% delayed)
4. Verify: column should have only 0 and 1, no nulls
5. Log class distribution percentages

In [ ]:
# TODO: Analyze Min Delay distribution
# Calculate: min, max, mean, median, percentiles (25, 50, 75)
# Hint: df['Min Delay'].describe()

In [ ]:
# TODO: Decide on delay threshold
# Set variable: threshold = 5 (or your chosen value)
# Comment why you chose this threshold

In [ ]:
# TODO: Create is_delayed column
# If Min Delay >= threshold, then 1, else 0
# Hint: df['is_delayed'] = (df['Min Delay'] >= threshold).astype(int)

In [ ]:
# TODO: Check class distribution
# Count of 0s and 1s, and percentages
# Hint: df['is_delayed'].value_counts() and normalized version

In [ ]:
# TODO: Validate target variable
# Check: only 0 and 1 values, no nulls, data type is int

---

## Task 6: Create Day-of-Week Numeric

**Objective:** Convert day names (Monday, Tuesday, etc.) to numeric (0-6)

**Mapping:**
- Monday → 0
- Tuesday → 1
- ... 
- Sunday → 6

**Hints:**
1. Create mapping dictionary with day names as keys
2. Use `.map()` to apply mapping
3. Verify: values should be 0-6 only, no nulls
4. Spot-check: verify a few rows match expected days

In [ ]:
# TODO: Check current Day column
# See unique values
# Hint: df['Day'].unique()

In [ ]:
# TODO: Create day mapping dictionary
# Map day names to 0-6
# Example: {'Monday': 0, 'Tuesday': 1, ...}

In [ ]:
# TODO: Create day_of_week column
# Apply mapping to Day column
# Hint: df['day_of_week'] = df['Day'].map(day_mapping)

In [ ]:
# TODO: Validate day_of_week
# Check: values 0-6 only, no nulls, data type is int
# Spot-check a few rows

---

## Task 7: Create Weekday Indicator

**Objective:** Create binary column 'is_weekend' (0=Mon-Fri, 1=Sat-Sun)

**Logic:**
- If day_of_week >= 5 (Saturday or Sunday), then 1
- Else 0

**Hints:**
1. Use comparison operator: `df['day_of_week'] >= 5`
2. Convert boolean to int with `.astype(int)`
3. Verify: only 0 and 1 values
4. Spot-check: Saturday and Sunday rows should be 1

In [ ]:
# TODO: Create is_weekend column
# 1 if day_of_week >= 5, else 0
# Hint: df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

In [ ]:
# TODO: Validate is_weekend
# Check: only 0 and 1 values, no nulls
# Spot-check: verify Saturday/Sunday are marked as 1

---

## Task 8: Calculate Historical Frequencies

**Objective:** Create features showing historical delay patterns

**Three features to create:**
1. **route_delay_frequency** - % of delays by Line
2. **route_hour_delay_frequency** - % of delays by Line + Hour
3. **route_day_hour_delay_frequency** - % of delays by Line + Day + Hour

**Hints:**
1. Group by Line, calculate mean of is_delayed (gives proportion)
2. Create mapping: Line → frequency value
3. Map back to all rows with the same Line
4. Repeat for Line+Hour and Line+Day+Hour combinations
5. All values should be between 0 and 1 (proportions)
6. Verify: no nulls in any frequency columns

In [ ]:
# TODO: Calculate route-level delay frequency
# Group by Line, calculate mean of is_delayed
# Create mapping and apply to all rows
# Store in column: route_delay_frequency
# Hint: group_by -> mean -> map

In [ ]:
# TODO: Calculate route-hour delay frequency
# Group by Line + Hour combination
# Calculate mean of is_delayed
# Map back to all rows
# Store in column: route_hour_delay_frequency
# Hint: Create composite key first: Line + '_' + hour

In [ ]:
# TODO: Calculate route-day-hour delay frequency
# Group by Line + Day + Hour combination
# Calculate mean of is_delayed
# Map back to all rows
# Store in column: route_day_hour_delay_frequency
# Hint: Create composite key: Line + '_' + day_of_week + '_' + hour

In [ ]:
# TODO: Validate frequency columns
# Check: all values between 0 and 1
# Check: no nulls
# Sample a few rows to verify values make sense

---

## Task 9: Handle Missing Bound Column

**Objective:** Decide what to do with Bound column (has ~36% missing values)

**Decision Point:**
- **Option A (Recommended): Drop the column** - Simpler, not needed for Phase 1
- Option B: Keep and note the missing values

**Hints:**
1. Check current Bound values: `df['Bound'].unique()` and null count
2. If choosing Option A: use `df.drop(columns=['Bound'])`
3. If choosing Option B: create `has_bound` indicator (optional)

In [ ]:
# TODO: Examine Bound column
# Check unique values, null count, percentage
# Hint: df['Bound'].value_counts(dropna=False)

In [ ]:
# TODO: Drop Bound column
# Since not needed for Phase 1
# Hint: df = df.drop(columns=['Bound'])
# OR: Keep it if you prefer

---

## Task 10: Final Validation

**Objective:** Verify all preprocessing is complete and correct

**Validation Checklist:**
1. No null values in critical columns (Line, hour, day_of_week, is_delayed, is_weekend)
2. Data types correct (int, float, datetime, object)
3. Value ranges correct (hour 0-23, day_of_week 0-6, frequencies 0-1)
4. Record counts reasonable (~52,000-52,064)
5. No unexpected data loss

**Hints:**
1. Use `df.info()` to check data types and nulls
2. Use `df.describe()` to check numeric ranges
3. Create summary table showing column info

In [ ]:
# TODO: Check for null values
# List all columns with null counts
# Hint: df.isnull().sum()

In [ ]:
# TODO: Check data types
# Verify each column has expected type
# Expected:
#   - Line: object
#   - hour, day_of_week, is_weekend, is_delayed: int
#   - Frequencies: float
#   - Date: datetime
# Hint: df.dtypes

In [ ]:
# TODO: Check value ranges
# hour: 0-23
# day_of_week: 0-6
# is_weekend: 0-1
# is_delayed: 0-1
# month: 1-12
# Frequencies: 0-1
# Hint: df.describe() and custom checks

In [ ]:
# TODO: Check record counts by line
# Breakdown of records for each line
# Should see: Line 1, Line 2, Line 4, etc.
# Hint: df['Line'].value_counts()

In [ ]:
# TODO: Generate validation summary
# Create table showing:
#   - Column name
#   - Data type
#   - Null count
#   - Min/Max (for numeric)
# Hint: Create DataFrame with this info

In [ ]:
# TODO: Final checks
# - Total record count
# - Date range (should be 2024-2025)
# - No unexpected patterns
# Print summary statement: "Validation passed" or list issues

---

## Task 11: Export Clean Data

**Objective:** Save preprocessed data to CSV for modeling

**Output File:** `cleaned_ttc_delay_data.csv`

**Columns to Include:**
- Date, Line, hour, day_of_week, is_weekend
- month, week, year
- Min Delay (original)
- is_delayed (TARGET VARIABLE)
- route_delay_frequency, route_hour_delay_frequency, route_day_hour_delay_frequency
- Code (delay reason, for exploration)
- Station (location, for exploration)

**Hints:**
1. Select specific columns to keep
2. Use `to_csv()` with appropriate parameters
3. Read file back in to verify export was successful
4. Spot-check 5-10 random rows

In [ ]:
# TODO: Select columns to keep
# Create list of column names to export
# Include all required columns from task guide

In [ ]:
# TODO: Create final dataframe with selected columns
# Hint: df_export = df[selected_columns]

In [ ]:
# TODO: Export to CSV
# File: cleaned_ttc_delay_data.csv
# Parameters:
#   - index=False
#   - encoding='utf-8'
# Hint: df_export.to_csv('cleaned_ttc_delay_data.csv', index=False)

In [ ]:
# TODO: Verify export
# Read file back in
# Check shape, columns, data types
# Hint: df_check = pd.read_csv('cleaned_ttc_delay_data.csv')

In [ ]:
# TODO: Spot-check exported data
# Display 5-10 random rows
# Verify data looks correct
# Hint: df_check.sample(10)

In [ ]:
# TODO: Final summary
# Print completion message
# Example: "✓ Preprocessing complete! Exported 52,064 records to cleaned_ttc_delay_data.csv"

---

## Notes

- Follow tasks in order (1-11)
- After each task, validate before moving to next
- When code is ready, share with me for review
- Ask questions if hints are unclear
- Final output: `cleaned_ttc_delay_data.csv` ready for Phase 1 modeling